In [1]:
from lstm_architecture import one_hot, run_with_config
import numpy as np
import os

# network config

In [2]:
class Config(object):
    """
    define a class to store parameters,
    the input should be feature mat of training and testing
    """

    def __init__(self, X_train, X_test):
        # Data shaping
        self.train_count = len(X_train)  # 7352 training series
        self.test_data_count = len(X_test)  # 2947 testing series
        self.n_steps = len(X_train[0])  # 128 time_steps per series
        self.n_classes = 7  # Final output classes

        # Training
        self.learning_rate = 0.001
        self.lambda_loss_amount = 0.005
        self.training_epochs = 100
        self.batch_size = 100
        self.clip_gradients = 15.0
        self.gradient_noise_scale = None
        # Dropout is added on inputs and after each stacked layers (but not
        # between residual layers).
        self.keep_prob_for_dropout = 0.85  # **(1/3.0)

        # Linear+relu structure
        self.bias_mean = 0.3
        # I would recommend between 0.1 and 1.0 or to change and use a xavier
        # initializer
        self.weights_stddev = 0.2

        ########
        # NOTE: I think that if any of the below parameters are changed,
        # the best is to readjust every parameters in the "Training" section
        # above to properly compare the architectures only once optimised.
        ########

        # LSTM structure
        # Features count is of 9: three 3D sensors features over time
        self.n_inputs = len(X_train[0][0])
        self.n_hidden = 28  # nb of neurons inside the neural network
        # Use bidir in every LSTM cell, or not:
        self.use_bidirectionnal_cells = False

        # High-level deep architecture
        self.also_add_dropout_between_stacked_cells = False 

# load data

In [3]:
def load_X(X_path):
    X_list = []
    file = open(X_path, 'r')
    # Read dataset from disk, dealing with text files' syntax
    X_signal = [np.array(item, dtype=np.float32) for item in [
               line.strip().split('\t') for line in file]]
    X_list.append(X_signal)
    file.close()
    return np.transpose(np.array(X_list), (1, 2, 0))


# Load "y" (the neural network's training and testing outputs)
def load_y(y_path):
    file = open(y_path, 'r')
    # Read dataset from disk, dealing with text file's syntax
    y_ = np.array([elem for elem in [line.strip().split('\t') for line in file]], 
                  dtype=np.int32)
    file.close()
    # Substract 1 to each output class for friendly 0-based indexing
    return one_hot(y_)

labels = ['info', 'crit', 'err', 'notice', 'warning', 'alert', 'emerg']

dataset_path = "data/"
print("\n" + "Dataset is now located at: " + dataset_path)

X_train_signals_path = dataset_path + "msg_token_train.txt"
X_test_signals_path = dataset_path + "msg_token_test.txt"
X_train = load_X(X_train_signals_path)
X_test = load_X(X_test_signals_path)

y_train_path = dataset_path + "msg_label_train.txt"
y_test_path = dataset_path + "msg_label_test.txt"

y_train = load_y(y_train_path)
y_test = load_y(y_test_path)
print(X_train.shape, y_train.shape, y_test.shape)


Dataset is now located at: data/
((8360, 120, 1), (8360, 7), (1671, 7))


# train and test

In [4]:
n_layers_in_highway = 0
n_stacked_layers = 3
trial_name = "{}x{}".format(n_layers_in_highway, n_stacked_layers)

for learning_rate in [0.001]:  # [0.01, 0.007, 0.001, 0.0007, 0.0001]:
    for lambda_loss_amount in [0.005]:
        for clip_gradients in [15.0]:
            print "learning_rate: {}".format(learning_rate)
            print "lambda_loss_amount: {}".format(lambda_loss_amount)
            print ""

            class EditedConfig(Config):
                def __init__(self, X, Y):
                    super(EditedConfig, self).__init__(X, Y)

                    # Edit only some parameters:
                    self.learning_rate = learning_rate
                    self.lambda_loss_amount = lambda_loss_amount
                    self.clip_gradients = clip_gradients
                    # Architecture params:
                    self.n_layers_in_highway = n_layers_in_highway
                    self.n_stacked_layers = n_stacked_layers

            # # Useful catch upon looping (e.g.: not enough memory)
            # try:
            #     accuracy_out, best_accuracy = run_with_config(EditedConfig)
            # except:
            #     accuracy_out, best_accuracy = -1, -1
            accuracy_out, best_accuracy, f1_score_out, best_f1_score = (
                run_with_config(EditedConfig, X_train, y_train, X_test, y_test)
            )
            print (accuracy_out, best_accuracy, f1_score_out, best_f1_score)

            with open('{}_result_.txt'.format(trial_name), 'a') as f:
                f.write(str(learning_rate) + ' \t' + str(lambda_loss_amount) + \
                        ' \t' + str(clip_gradients) + ' \t' + str(accuracy_out) + \
                        ' \t' + str(best_accuracy) + ' \t' + str(f1_score_out) + \
                        ' \t' + str(best_f1_score) + '\n\n')

            print "______________________________________"
        print ""
print "Done."

learning_rate: 0.001
lambda_loss_amount: 0.005

Some useful info to get an insight on dataset's shape and normalisation:
features shape, labels shape, each features mean, each features standard deviation
((1671, 120, 1), (1671, 7), 19643.785, 5585.6108)
the dataset is therefore properly normalised, as expected.
(120, ?, 1)
(?, 1)
(120, '(?, 1)')

Creating hidden #1:
(120, '(?, 28)')

Creating hidden #2:
(120, '(?, 28)')

Creating hidden #3:
(120, '(?, 28)')

Unregularised variables:
LSTM_network/layer_1/relu_fc_biases_noreg:0
LSTM_network/layer_2/relu_fc_biases_noreg:0
LSTM_network/layer_3/relu_fc_biases_noreg:0
LSTM_network/relu_fc_biases_noreg:0


/usr/local/lib/python2.7/dist-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


iter: 0, train loss: 1.44182527065, train accuracy: 0.459999978542, train F1-score: 0.435858585859, test loss: 1.57855677605, test accuracy: 0.415320128202, test F1-score: 0.243748157539
iter: 1, train loss: 1.24170589447, train accuracy: 0.459999978542, train F1-score: 0.389556451613, test loss: 1.41779947281, test accuracy: 0.436265677214, test F1-score: 0.265031418312
iter: 2, train loss: 1.15945875645, train accuracy: 0.519999980927, train F1-score: 0.40947295423, test loss: 1.34869003296, test accuracy: 0.436265677214, test F1-score: 0.265031418312
iter: 3, train loss: 1.12953484058, train accuracy: 0.42999997735, train F1-score: 0.383257918552, test loss: 1.31111299992, test accuracy: 0.436265677214, test F1-score: 0.265031418312
iter: 4, train loss: 1.0831694603, train accuracy: 0.479999989271, train F1-score: 0.382935779817, test loss: 1.28837275505, test accuracy: 0.436265677214, test F1-score: 0.265031418312
iter: 5, train loss: 1.05748581886, train accuracy: 0.490000009537, 

KeyboardInterrupt: 